In [1]:
import requests
from bs4 import BeautifulSoup

In [102]:
import pandas as pd

In [2]:
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }


In [3]:
r = requests.get(SITE, headers = HEADERS)


In [4]:
soup = BeautifulSoup(r.text, 'lxml')

In [112]:
# convert coordinates in degrees to float
import re
def conv_coordinates(lat):
    deg, minutes, seconds, direction =  re.split('[°\′″]', lat)
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)

In [32]:
# check the function
SITE = 'https://en.wikipedia.org/wiki/Zhytomyrska_(Kyiv_Metro)'

In [33]:
r = requests.get(SITE, headers = HEADERS)

In [34]:
soup = BeautifulSoup(r.text, 'lxml')

In [39]:
lat = soup.find('span',class_ = 'latitude')
lat.text

'50°27′21″N'

In [40]:
conv_coordinates(lat.text)

50.45583333333334


In [38]:
soup.find('span',class_ = 'longitude')

<span class="longitude">30°21′51″E</span>

In [41]:
SITE = 'https://en.wikipedia.org/wiki/Sviatoshynsko%E2%80%93Brovarska_line'

In [42]:
r = requests.get(SITE, headers = HEADERS)
soup = BeautifulSoup(r.text, 'lxml')

In [59]:
metro_station = []
for li in soup.find_all('li'):
    try:
        metro = li.a.get('href')
        if '(Kyiv_Metro)'in metro:
            metro_station.append(metro)
    except:
        None
print(len(metro_station))

72


In [60]:
metro_link = set(metro_station)

In [85]:
metro_link = list(metro_link)

In [115]:
SITE = 'https://en.wikipedia.org'
metro_lat = []
for i in metro_link:
    r = requests.get(SITE+i, headers=HEADERS)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        el1 = soup.find('span', class_='latitude').text
        el2 = soup.find('span', class_='longitude').text
        station = soup.find('h1', class_='firstHeading').text
        el1 = conv_coordinates(el1)
        el2 = conv_coordinates(el2)
        
        
        coord = (station,el1,el2)
        
        metro_lat.append(coord)
metro_lat

[('Sviatoshyn (Kyiv Metro)', 50.4575, 30.391944444444444),
 ('Vystavkovyi Tsentr (Kyiv Metro)', 50.3825, 30.4775),
 ('Khreshchatyk (Kyiv Metro)', 50.447222222222216, 30.522777777777776),
 ('Dorohozhychi (Kyiv Metro)', 50.47361111111111, 30.449166666666667),
 ('Syrets (Kyiv Metro)', 50.47638888888889, 30.430833333333336),
 ('Kontraktova Ploshcha (Kyiv Metro)', 50.46527777777778, 30.516666666666666),
 ('Zoloti Vorota (Kyiv Metro)', 50.44833333333333, 30.513333333333332),
 ('Vokzalna (Kyiv Metro)', 50.44166666666666, 30.488055555555558),
 ('Maidan Nezalezhnosti (Kyiv Metro)', 50.45, 30.524444444444445),
 ('Palats Sportu (Kyiv Metro)', 50.43805555555555, 30.520833333333332),
 ('Chervonyi Khutir (Kyiv Metro)', 50.40888888888889, 30.694444444444446),
 ('Beresteiska (Kyiv Metro)', 50.45861111111111, 30.419722222222223),
 ('Vyrlytsia (Kyiv Metro)', 50.40333333333333, 30.66611111111111),
 ('Pochaina (Kyiv Metro)', 50.48694444444445, 30.497777777777777),
 ('Teremky (Kyiv Metro)', 50.367044444444

In [117]:
metro = pd.DataFrame(metro_lat)

In [118]:
import pickle
with open('metro.pickle', 'wb') as f:     #pickle працює в бінарному режимі 
    pickle.dump(metro, f)

In [97]:
len(metro_lat),len(metro_link)

(54, 54)